In [ ]:
#UP2009045
import flwr as fl
import LR_Utils as utils
from flwr.common import NDArrays, Scalar
from sklearn.metrics import log_loss, ConfusionMatrixDisplay
from sklearn import metrics
import pandas as pd
from sklearn.linear_model import LogisticRegression
from typing import Tuple, Dict, Optional
import numpy as np

In [ ]:
def fit_round(server_round: int) -> Dict:
    """Send round number to client."""
    return {"server_round": server_round}

In [ ]:
def get_evaluate_fn(model: LogisticRegression):
    """Return an evaluation function for server-side evaluation."""

    (X_train,y_train),(X_test, y_test) = utils.load_Data()

    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        utils.set_model_params(model, parameters)
        #loss = log_loss(y_test, X_test)
        accuracy = model.score(X_test, y_test)
        return {"Server Accuracy": accuracy}
        
    return evaluate

In [ ]:
if __name__ == "__main__":
    model = LogisticRegression()
    utils.set_initial_params(model)
    strategy = fl.server.strategy.FedAvg(
        min_available_clients=2,
        evaluate_fn=get_evaluate_fn(model),
        on_fit_config_fn=fit_round,

    )

In [ ]:
fl.server.start_server(
    server_address="127.0.0.1:8080",
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=300)
)